In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.custom_generator_NEW import CustomGeneratorNEW
from tohu.utils import print_generated_sequence

## Custom generator without `__init__` method

The simplest way to use `CustomGenerator` is with only class attributes (without `__init__` method).

In [2]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    cc = FakerGenerator(method="name")

In [3]:
g = QuuxGenerator()

2020-04-23 09:04:46 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [4]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


Note that while the _instance_ `g` contains attributes with the same names `"aa"` and `"bb"` as the parent class `QuuxGenerator`, internally the instance attributes are actually spawned versions of the class attributes (in particular, they are not identical):

In [5]:
print(QuuxGenerator.aa)
print(g.aa)
assert QuuxGenerator.aa is not g.aa

print(QuuxGenerator.bb)
print(g.bb)
assert QuuxGenerator.bb is not g.bb

print(QuuxGenerator.cc)
print(g.cc)
assert QuuxGenerator.cc is not g.cc

<Integer (id=b341f9)>
<Integer (id=76725b)>
<HashDigest (id=3d1413)>
<HashDigest (id=e3ace0)>
<FakerGenerator (id=f49ce8)>
<FakerGenerator (id=d11877)>


## Custom generator with `__init__` method

Tohu generators that are defined as instance attributes within the `__init__` method are also picked up.

In [6]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    
    def __init__(self, method):
        self.cc = FakerGenerator(method=method)

In [7]:
g = QuuxGenerator(method="first_name")

2020-04-23 09:04:47 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [8]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Mary')
Quux(aa=171, bb='54596E', cc='Jennifer')
Quux(aa=142, bb='2A16D0', cc='Christopher')
Quux(aa=140, bb='FDCDD3', cc='Joseph')
Quux(aa=121, bb='BDE283', cc='Kelly')


---

## Looped custom generator

In [9]:
from tohu.foreach_NEW import foreach_NEW

In [10]:
@foreach_NEW(xx=["AAA", "BBB"])
@foreach_NEW(yy=[111, 222, 333])
class QuuxGenerator(CustomGeneratorNEW):
    aa = xx
    bb = yy
    cc = Integer(100, 200)
    dd = HashDigest(length=6)

Internally, the `QuuxGenerator` class has been augmented by the `@foreach_NEW` decorators with information about the loop level (here: 2, since we have two nested decorators) and the loop variables present.

In [11]:
QuuxGenerator._tohu_loop_level

2

Loop variables:

In [12]:
QuuxGenerator._tohu_cg_class_loop_variables

[<LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=d4afb8)>,
 <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=f186d5)>]

In [13]:
g = QuuxGenerator()

2020-04-23 09:04:47 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [14]:
g._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=628c93)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=829a70)>}

In [15]:
g._loop_runner.loop_variables["xx"].reset_loop_variable()
print_generated_sequence(g, num=5, seed=11111, sep="\n")
g._loop_runner.loop_variables["xx"].advance()
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa='AAA', bb=111, cc=146, dd='9CD736')
Quux(aa='AAA', bb=111, cc=192, dd='3A7BFE')
Quux(aa='AAA', bb=111, cc=123, dd='89AA37')
Quux(aa='AAA', bb=111, cc=110, dd='8B408D')
Quux(aa='AAA', bb=111, cc=134, dd='BB048F')
Generated sequence:

Quux(aa='AAA', bb=111, cc=146, dd='9CD736')
Quux(aa='AAA', bb=111, cc=192, dd='3A7BFE')
Quux(aa='AAA', bb=111, cc=123, dd='89AA37')
Quux(aa='AAA', bb=111, cc=110, dd='8B408D')
Quux(aa='AAA', bb=111, cc=134, dd='BB048F')


In [16]:
#g._loop_runner.loop_variables["xx"].advance()

Both the parent class `QuuxGenerator` and its instance `g` contain loop runners containing loop variables `xx` and `yy`.

In [17]:
QuuxGenerator._tohu_cg_class_loop_variables

[<LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=d4afb8)>,
 <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=f186d5)>]

In [18]:
g._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=628c93)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='BBB' (tohu_id=829a70)>}

In [19]:
assert g._loop_runner.loop_variables["xx"].is_clone_of(QuuxGenerator._tohu_cg_class_loop_variables[1])
assert g._loop_runner.loop_variables["yy"].is_clone_of(QuuxGenerator._tohu_cg_class_loop_variables[0])